![Empowering Picture](intro.png)

# Das Garagentor

## Lerninhalte

Diese Aufgabe besteht aus mehreren Teilen und beinhaltet nachfolgende Lerninhalte:
  - Grundwissen Python: Parsen von Strings in float, Arrays, Tuples, Generatoren, usw. 
  - [MQTT](https://de.wikipedia.org/wiki/MQTT), siehe Youtube Video [What is MQTT and How It Works](https://www.youtube.com/watch?v=EIxdz-2rhLs) 
  - MQTT Client mit der Library [paho-mqtt](https://pypi.org/project/paho-mqtt/)
  - Informationen mit einem [Flask](https://flask.palletsprojects.com/en/3.0.x/) [HTTP Server](https://de.wikipedia.org/wiki/Client-Server-Modell) bereitstellen
  - Notifications mit [Pushover](https://pushover.net/) auf ein Smartphone schicken 
  - Mit einem [Scheduler](https://schedule.readthedocs.io/en/stable/index.html) regelmässig eine Funktion (Task) ausführen

## Voraussetzungen
Nebst den Standard-Libraries von Python werden folgende zusätzliche Libraries benötigt (Tipp: Verwende `requirements.txt` für die Installation aller Depedencies in Pycharm):
 - paho-mqtt
 - requests
 - schedule
 - flask

Wenn du einen physischen Garagesensor zum Testen verwenden möchtest und einen M5Stack Core2 zur Verfügung hast, kannst du folgende Firmware verwenden: [alptbz/core2imu](https://github.com/alptbz/core2imu)

## Intro

<i>Du lebst mit Freunden in einem kleinen Haus am Stadtrand. Alles läuft rund und ihr habt es gemütlich zusammen. Wäre da nicht das kleine Drama mit dem Garagentor: Einer deiner Freunde lässt immer das Garagentor offen, wenn er am Morgen mit seinem Cargo-Velo zur Arbeit fährt. Trotz mehreren Krisensitzungen in der Wohngemeinschaft hat sich sein Verhalten nicht wirklich verbessert. Jetzt vergisst er es nicht mehr jedes Mal, aber jedes zweite Mal. Es wurde bereits eine Spitzhacke aus der Garage gestohlen und eine Katze hat hineingekotet. Du hast genug davon und möchtest mit einer technischen Lösung deinen Mitbewohner daran erinnern das Garagentor zu schliessen, wenn er mal wieder nicht selber daran denkt. Hoffentlich vergisst er sein Handy nicht und prüft es regelmässig auf neue Nachrichten...</i> 

<i>Im ersten Schritt benötigst du einen Sensor, der zuverlässig feststellen kann, ob das Garagentor offen ist oder nicht. Du entscheidest dich dies mit einem Lagesensor umzusetzen. Als Informatiker hast du selbstverständlich ein perfektes WLAN im ganzen Haus. Deshalb entscheidest du dich für einen WLAN basierten Lagesensor der MQTT als Kommunikationsprotokoll unterstützt.</i>

![Lagesensor am Garagentor](garagentor_schema.png)

<i>Abbildung: Der Lagesensor wird direkt am Tor angebracht. Sobald sich das Tor öffnet, ändert der Lagesensor seine Orientierung im Raum und damit die Messwerte des Beschleunigungssensors. </i>

In dieser Übung werden wir Schritt für Schritt ein Backend bauen, dass die Sensorwerte via MQTT entgegennimmt, auswertet und wenn das Garagentor zu lange offen steht, eine Benachrichtigung an ein Smartphone sendet. 

Zu Beginn der Übung benötigst du nur dieses *Juypter Notebook*. 

# Teil 1 - Parsen

<i>Inzwischen hast du den Sensor gekauft, am Garagentor befestigt, per WLAN verbunden und mit dem MQTT Broker verknüpft. Obwohl der Sensor in der Lage ist selbstständig festzustellen, ob das Garagentor offen ist oder nicht und dies sogar auf dem Bildschirm anzeigt, verschickt er via MQTT nur seine Sensordaten des Beschleunigungssensors.</i>

Nun benötigen wir eine Funktion, die uns zuverlässig angibt, ob das Garagentor geöffnet ist oder nicht. 

- Beispielwert für das geschlossene Tor: `-0.03,0.99,0.05`
- Beispielwert für das geöffnete Tor: `-0.09,0.03,-0.92`

(Der Wert 1.0 entspricht der einfachen Erdbeschleunigung = 1 G. (~ 9.81 m/s^2).)  

Die Werte stehen für die Beschleunigungen der jeweiligen Achsen (x, y, z). Wenn der Sensor relativ zur Erde in Ruhe ist, entspricht die resultierende Vektorsumme der Beschleunigung der auf den Sensor wirkenden Gravitationsbeschleunigung der Erde.



<b>Challenge 1: Schreibe eine Funktion, die den `raw_sensor_value` in drei Teile aufsplittet und als Array zurückgibt.</b>

Beispiel Input: `"-0.03,0.99,0.05"` => Output: `['-0.03', '0.99', '0.05']`

In [18]:
def split_into_values(raw_sensor_value: str):
    # TODO: Implement
    values = raw_sensor_value.split(',')
    return values

split_into_values("-0.03,0.99,0.05")

['-0.03', '0.99', '0.05']

<b>Challenge 2: Erweitere die Funktion so, dass sie drei Zahlenwerte (`float`) als Tuple zurückgibt. </b>

Beispiel: Input: `"-0.03,0.99,0.05"` => Output: `(-0.03, 0.99, 0.05)`

In [19]:
def split_into_axis(raw_sensor_value: str):
    # TODO: Implement
    values_str = raw_sensor_value.split(',')
    values_float = [float(value) for value in values_str]
    values_tuple = tuple(values_float)
    return values_tuple
    pass

split_into_axis("-0.03,0.99,0.05")

(-0.03, 0.99, 0.05)

<b>Challenge 3: Nun entwickle eine weitere Funktion, die als Input den `raw_sensor_value` annimmt und dir zuverlässig angibt, ob das Tor offen (True), geschlossen (False) ist oder der Zustand unbekannt ist (None). </b>


In [20]:
def is_garage_door_open(raw_sensor_value: str):
    x, y, z = split_into_axis(raw_sensor_value)
    # TODO: Implement
    if z < -0.8:
        return True  # Garage door is open
    elif z > -0.1:
        return False  # Garage door is closed
    else:
        return None  # Unknown state
    pass
    
assert not is_garage_door_open("-0.03,0.99,0.05")
assert not is_garage_door_open("-0.03,0.85,0.05")
assert is_garage_door_open("-0.03,0.0,-0.85")
assert is_garage_door_open("-0.03,0.0,-0.9")
assert is_garage_door_open("-0.03,0.0,-0.4") is None
print("All tests successful")


All tests successful


<b>Challenge 4: Nun benötigen wir eine weitere Funktion, die als Input den Rückgabewert aus `is_garage_door_open` erhält und einen String mit dem jeweilig passenden Inhalt "Offen", "Geschlossen" oder "Wedernoch" zurückgibt.  </b>


In [21]:
def garage_door_eval_to_str(eval):
    # TODO: Implement
    if eval is True:
        return "Offen"
    elif eval is False:
        return "Geschlossen"
    else:
        return "Wedernoch"
    
    
assert garage_door_eval_to_str(True) == "Offen"
assert garage_door_eval_to_str(False) == "Geschlossen"
assert garage_door_eval_to_str(None) == "Wedernoch"
print("All tests successful")


All tests successful


<b>Challange 5: Die Verwendung von None als Zustand ist ein wenig unschön. Mit einem Enum können wir das besser im Code abbilden. Baue eine neue Funktion `eval_garage_door_state`, die als Rückgabewert ein Enum des Typs `GarageDoorState` zurückgibt. </b>

<i>Erklärung: `None` eignet sich hier nicht um einen Zustand abzubilden, weil es den Code schwerer verständlicher macht und sich `None` anders verhält. Mithilfe des Enums ist es für den Entwickler viel klarer, welcher Zustand das Tor gerade hat.</i> 

In [22]:
from enum import Enum

class GarageDoorState(Enum):
    CLOSED = 1,
    OPEN = 2,
    NEITHER = 3
    
def eval_garage_door_state(raw_sensor_value: str) -> GarageDoorState:
    x, y, z = split_into_axis(raw_sensor_value)
    # TODO: Implement
    if z < -0.8:
        return GarageDoorState.OPEN
    elif z > -0.1:
        return GarageDoorState.CLOSED
    else:
        return GarageDoorState.NEITHER

assert eval_garage_door_state("-0.03,0.99,0.05") == GarageDoorState.CLOSED
assert eval_garage_door_state("-0.03,0.85,0.05") == GarageDoorState.CLOSED
assert eval_garage_door_state("-0.03,0.0,-0.85") == GarageDoorState.OPEN
assert eval_garage_door_state("-0.03,0.0,-0.9") == GarageDoorState.OPEN
assert eval_garage_door_state("-0.03,0.0,-0.4")  == GarageDoorState.NEITHER
print("All tests successful")

All tests successful


<b>Challenge 6: Erweitere nun die Enum Klasse so, dass sie für einen Enum Wert einen String mit den Beschreibungen aus Challenge 4 zurückgibt. Ergänze den nachfolgenden Code:</b>

In [4]:
from enum import Enum

class GarageDoorState(Enum):
    CLOSED = 1,
    OPEN = 2,
    NEITHER = 3

    def __str__(self):
        if self.value == GarageDoorState.OPEN.value:
            return "Offen"
        # TODO: Implement
        elif self.value == GarageDoorState.CLOSED.value:
            return "Geschlossen"
        else:
            return "Wedernoch"
    
   
assert str(GarageDoorState.OPEN) == "Offen"
assert str(GarageDoorState.CLOSED) == "Geschlossen"
assert str(GarageDoorState.NEITHER) == "Wedernoch"

print("All tests successful")

All tests successful


# Teil 2 - MQTT

Jetzt sind wir in der Lage die Sensorwerte zu parsen und auszuwerten. Im nächsten Schritt möchten wir nun die echten Sensorwerte via MQTT empfangen und im Terminal ausgeben, wenn das Garagentor geöffnet ist und wann nicht. 

Dazu verwenden wir die Library [paho-mqtt](https://pypi.org/project/paho-mqtt/). In README findet man einen Beispielcode. Dieser liegt unten in angepasster Form vor. 

Es gibt unzählige Tools und Libraries, um sich mit einem MQTT Broker zu verbinden (Auch als `Client Library` bezeichnet). Ein nützliches Tool ist der [MQTT Explorer](https://mqtt-explorer.com/). 

Installiere dieses Programm auf deinem Gerät und verbinde ich anschliessend mit dem MQTT Server `cloud.tbz.ch` (Protokoll: mqtt, Port: 1883). Benutzername und Passwort wird keines benötigt.  

Anschliessend kannst du auf dem Topic, dass du unten im Code definiert hast (aktuell `m5core2/accel`), mithilfe des MQTT Explorer eine Message publishen und prüfen, ob diese ausgegeben wird. 

<b>Achtung: Wenn du die Zelle (Skript) unten startest, beendet sich das Skript nicht automatisch. `client.loop_forever()` macht, dass das Skript weiterläuft, sodass es Messages via MQTT empfangen und verarbeiten kann. Ob eine Zelle noch ausgeführt wird, kann an einem sich bewegenden schmalen Balken am Ende der Zelle erkannt werden. Du musst die Zelle bzw. das Skript jeweils von Hand stoppen.</b>

## MQTT Broker - cloud.tbz.ch

In dieser Übung wird der öffentliche MQTT Broker der TBZ verwenden (cloud.tbz.ch). Jeder kann sich mit diesem Broker ohne Authentifizierung verbinden. Dieser Broker eignet sich nur für Demo-Zwecke und ist nicht für produktive Anwendungen verwendet werden. Das Übermitteln von Personendaten über diesen Broker stellt eine Datenschutzverletzung dar und ist zu unterlassen. 

Es gibt im Internet unzähle Anleitungen, wie man einen eigenen MQTT Broker (=MQTT Server) aufsetzen kann:
 - [Raspberry Pi: MQTT-Broker Mosquitto installieren und konfigurieren](https://www.elektronik-kompendium.de/sites/raspberry-pi/2709041.htm)
 - [How to Install and Secure the Mosquitto MQTT Messaging Broker on Debian 10](https://www.digitalocean.com/community/tutorials/how-to-install-and-secure-the-mosquitto-mqtt-messaging-broker-on-debian-10)
 
Da alle Kursteilnehmer denselben MQTT Broker verwenden, empfiehlt es sich das Topic für den Sensorwert (`m5core2/accel`) individuell anzupassen. 

## Challenges

<b>Challenge 7: Setze nun deine Funktion `eval_garage_door_state` in den nachfolgenden Code ein, um die Sensorwerte auszuwerten und gebe das Resultat bzw. den Garangentor-Zustand auf dem Terminal aus.</b>

In [14]:
import paho.mqtt.client as mqtt

GARAGE_DOOR_TOPIC = "m5core2/accel"

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(GARAGE_DOOR_TOPIC)

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    if msg.topic == GARAGE_DOOR_TOPIC:
        msg_str = msg.payload.decode("utf-8")
        # TODO: Implement
        print(msg.topic, msg_str)    
    

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("cloud.tbz.ch", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()

Connected with result code 0
m5core2/accel -0.03,0.0,-0.85
m5core2/accel -0.03,0.0,-0.85


KeyboardInterrupt: 

<b>Challenge 8: Natürlich macht es keinen Sinn, dass jedes Mal, wenn der Sensor einen Wert übermittelt, auf dem Terminal der Zustand erneut ausgegeben wird. Erweitere den Code, dass nur bei Zustandsänderung eine Ausgabe erfolgt.</b> 

In [13]:
import paho.mqtt.client as mqtt

GARAGE_DOOR_TOPIC = "m5core2/accel"

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(GARAGE_DOOR_TOPIC)

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    global last_state
    if msg.topic == GARAGE_DOOR_TOPIC:
        msg_str = msg.payload.decode("utf-8")
        # TODO: Implement
        # Prüfen, ob sich der Zustand geändert hat
        if msg_str != last_state:
            # Zustandsänderung feststellen und ausgeben
            print(msg.topic, msg_str)
            
            # Vorherigen Zustand aktualisieren
            last_state = msg_str
        
    

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("cloud.tbz.ch", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()

Connected with result code 0
m5core2/accel -0.03,0.0,-0.85
m5core2/accel -0.03,0.0,-0.86
m5core2/accel -0.03,0.0,-0.85


Updated last_state: Geschlossen
Updated last_state: -0.03,0.0,-0.85
Updated last_state: -0.03,0.0,-0.85
Updated last_state: -0.03,0.0,-0.85
Updated last_state: -0.03,0.0,-0.85
Updated last_state: -0.03,0.0,-0.85
Updated last_state: -0.03,0.0,-0.85
Updated last_state: -0.03,0.0,-0.85
Updated last_state: -0.03,0.0,-0.85


KeyboardInterrupt: 

# Teil 3 - HTTP service

Nun möchten wir den Zustand des Garagentors über einen HTTP Service bereitstellen. Dafür verwenden wir das [HTTP Framework flask](https://flask.palletsprojects.com/en/3.0.x/). 

<b>Achtung: Wenn du die Zelle (Skript) unten startest, beendet sich das Skript nicht automatisch. `app.run()` macht, dass das Skript weiterläuft, sodass es HTTP Client Verbindungen entgegennehmen kann. Ob eine Zelle noch ausgeführt wird, kann an einem sich bewegenden schmalen Balken am Ende der Zelle erkannt werden. Du musst die Zelle bzw. das Skript jeweils von Hand stoppen.</b>

<b>Challenge 9: Probiere den nachfolgenden Flask Demo Code aus. Versuche eine weitere Unterseite `/hihi` hinzufügen, welche `Ente` zurückgibt. </b>

In [ ]:

# import flast module
from flask import Flask
 
# instance of flask application
app = Flask(__name__)
 
# home route that returns below text when root url is accessed
@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

# TODO: Implement new site /hihi
@app.route("/hihi")
def hihi():
    return "<h1>Ente</h1>"
 
if __name__ == '__main__':  
   app.run()

## MQTT Client mit HTTP Service

Der nachfolgende Code verknüpft den MQTT Client mit dem HTTP Server Service. 

<b>Challenge 10: Ergänze den nachfolgenden Code, sodass der Zustand des Garagentors via HTTP (d.h. über den Webbrowser) abgerufen werden kann.</b> 

In [32]:
import paho.mqtt.client as mqtt
from flask import Flask

# Topic on which we receive the sensor data
GARAGE_DOOR_TOPIC = "m5core2/accel"

# Flask application instance
app = Flask(__name__)

# global variable to keep the last state of the garage door 
#last_state: GarageDoorState = GarageDoorState.NEITHER

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect, then subscriptions will be renewed.
    client.subscribe(GARAGE_DOOR_TOPIC)

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    # Import the global variable into the function
    global last_state
    # Check if the topic is equal to ours
    if msg.topic == GARAGE_DOOR_TOPIC:
        msg_str = msg.payload.decode("utf-8")
        new_state = eval_garage_door_state(msg_str)
        # TODO: Implement
        last_state = new_state.name  

        print(f"Updated last_state: {last_state}")

# MQTT client setup
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.connect("cloud.tbz.ch", 1883, 60)

# Flask route definition
@app.route("/")
def index():
    return f"<h1>Die Garage ist {last_state}</h1>"

# MQTT client loop start
client.loop_start()

# Flask application run
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Connected with result code 0


127.0.0.1 - - [24/Jan/2024 18:12:37] "GET / HTTP/1.1" 200 -


Updated last_state: CLOSED
Updated last_state: CLOSED


127.0.0.1 - - [24/Jan/2024 18:12:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2024 18:12:56] "GET /favicon.ico HTTP/1.1" 404 -


Updated last_state: OPEN


127.0.0.1 - - [24/Jan/2024 18:13:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2024 18:13:28] "GET /favicon.ico HTTP/1.1" 404 -


Updated last_state: OPEN


Exception in thread Thread-122:
Traceback (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/fabiobeti/Documents/GitHub/FAAS/tasks/ipynb/mqtt/env/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/fabiobeti/Documents/GitHub/FAAS/tasks/ipynb/mqtt/env/lib/python3.9/site-packages/paho/mqtt/client.py", line 3591, in _thread_main
    self.loop_forever(retry_first_connection=True)
  File "/Users/fabiobeti/Documents/GitHub/FAAS/tasks/ipynb/mqtt/env/lib/python3.9/site-packages/paho/mqtt/client.py", line 1756, in loop_forever
    rc = self._loop(timeout)
  File "/Users/fab

# Teil 4 - Pushover

Natürlich wäre es toll, wenn wir automatisch eine Benachrichtigung auf unserem Smartphone erhalten, sobald das Garagentor zu lange offen steht. Um Benachrichtigungen auf ein Smartphone zu senden, gibt es verschiedene Möglichkeiten: E-Mail, Telegram Bot, usw. 

In diesem Beispiel verwenden wir den Service [Pushover](https://pushover.net). Dafür müssen wir:
- Auf [pushover.net](https://pushover.net) einen Account anlegen (Falls noch nicht vorhanden). 
- Pushover auf eigenem Smartphone installieren und einloggen
- Eigene Applikation registrieren (z.B. Demo Application), Siehe: [Pushover Application Registration](https://pushover.net/api#registration)<br>Achtung: Token wird nur einmal angezeigt!
- User Token und Application Token unten im Beispielcode eingeben und ausprobieren. 

<b>Challenge 11: Führe die oben aufgeführte Schritte aus und sende dir mit dem Beispielcode unten eine Message auf dein Smartphone</b>



In [ ]:
PUSHOVER_API_TOKEN = "a5qmdfh6owphuw3zwyobofhba7h1dw"
PUSHOVER_USER_ID = "ux23t17r4m9tv4mtjt2w45mfcp1bzg"

def send_message_to_me(message):
    import requests
    resp = requests.post("https://api.pushover.net/1/messages.json", data={"token": PUSHOVER_API_TOKEN , "user": PUSHOVER_USER_ID, "message": message})
    print(resp)

# Uncomment the following line to test:
send_message_to_me("Hello von Python!")

Sobald du die obige Zelle ausgeführt hast, kannst du die Funktion `send_message_to_me` auch an anderen Stellen in diesem Juypter Notebook verwenden. 

**Bevor du weiterfährst: Kopiere dir den Code von Challenge 10 in ein separates Python-Skript.**

Teilweise gibt es bei der Ausführung von Zellen, die länger im Hintergrund laufen, seltsame Probleme (z.B. läuft der Code weiter, aber es kommt kein Output mehr). 

<b>Challenge 12: Erweitere das Skript von Challenge 10, sodass es eine Meldung via *Pushover* abschickt, sobald sich der Zustand des Garagentors ändert. Den Zustand `NEITHER` soll nicht abgeschickt, sondern ignoriert werden.</b>

# Teil 5 - Logik

<i>Natürlich wollen wir nicht ständig informiert werden, ob das Garagentor geöffnet oder geschlossen wird. Stattdessen soll unser liebe(r) Mitbewohner*in eine Nachricht auf sein Smartphone erhalten, wenn das Garagentor zu lange offen steht. Ob er es gewesen ist oder nicht, ist uns egal. Die anderen Mitbewohner sind nämlich in der Lage das Garagentor zu schliessen und brauchen dazu keine extra Einladung. Die sind es sicher nicht gewesen. Es muss es/sie gewesen sein!</i>

Dafür verwenden wir folgende Logik:
- Sobald das Garagentor den Zustand auf `OPEN` wechselt, wird in einer Variable `last_opened_timestamp` der aktuelle Zeitpunkt (Datum + Uhrzeit) gespeichert und die Variable `notification_sent` auf `False` gesetzt. 
- Eine Funktion, welche alle 5 Sekunden ausgeführt wird, prüft, ob seit dem Zeitpunkt (`last_opened_timestamp`) mehr als 5 Minuten vergangen sind und ob `notification_sent` gleich `False` ist. (Zu Testzwecken kürzeren Zeitraum wählen). 
- Wenn ja, wird die Funktion `send_message_to_me` mit einer Nachricht `Schliess das verdammte Garagentor! Wir haben dich Lieb. Deine Mitbewohner.` aufgerufen und die Variable `notification_sent` auf `True` gesetzt, sodass erst wieder eine Notification abgeschickt wird, wenn das Garagentor in der Zwischenzeit geschlossen wurde.

<b>Achtung: Auch dieser Code läuft unendlich lang weiter, wenn er nicht gestoppt wird (oder dem Notebook der Strom ausgeht). Versuche herauszufinden weshalb!</b> 

<b>Challenge 13: Erweitere den nachfolgenden Code so, dass alle 5 Sekunden das Wort "Buuuum" ausgegen wird. Tipp: Es benötigt eine weitere Funktion!</b>

In [25]:
import schedule
import time

def job():
    print("Peng")

def job1():
    print("Boom")
    
schedule.clear()
schedule.every(2).seconds.do(job)
schedule.every(5).seconds.do(job1)

while True:
    schedule.run_pending()
    time.sleep(1)

Peng


KeyboardInterrupt: 

Das Beispiel mit "Peng" und "Buuum" ist schon noch "herzig", aber entspricht ja nicht ganz dem, was wir benötigen. In der nächsten Zelle ist die Logik mithilfe der Library `schedule` teilweise implementiert, welche wir für unser Garagentor brauchen. 

<b>Challenge 14: Ergänze den Code, sodass die Notification nur einmal abgeschickt wird (bzw. `Sending Notification` nur einmal auf der Konsole ausgegeben wird, unabhängig davon wie lange der Code läuft). (Anstatt fünf Minuten warten wir nur 5 Sekunden</b> 

In [26]:
import schedule
import time
from datetime import datetime, timedelta

last_state: GarageDoorState = GarageDoorState.OPEN
last_opened_timestamp = datetime.now()
notification_sent = False


def check_garage_door_state():
    global last_opened_timestamp, notification_sent
    if (last_state == GarageDoorState.OPEN 
            and datetime.now() - last_opened_timestamp > timedelta(seconds=5) 
            and not notification_sent):
        print("Sending notification")
        # TODO: Implement
        notification_sent = True


schedule.clear()
schedule.every(1).seconds.do(check_garage_door_state)

while True:
    schedule.run_pending()
    time.sleep(1)

Sending notification


KeyboardInterrupt: 

Nun haben wir alle Elemente, die wir für unseren Garagentor-Überwachung (oder Lazy-Friend-Helper) benötigen:
- Die Sensordaten werden geparst und der Zustand des Garagentors ermittelt. 
- Die Sensordaten werden per MQTT empfangen.
- Der Zustand des Garagentors kann über HTTP abgefragt werden.
- Via Pushover kann eine Nachricht auf ein Smartphone geschickt werden. 
- Ein Code ist in der Lage festzustellen, ob das Garagentor zu lange offengestanden ist. 

Nun müssen wir nur noch den letzten Punkt (die "Business-Logik") in unsere kleine Applikation einbauen.   

<b>Challenge 15: Implementiere nun die obige Logik in deinen Skript aus Challenge 12 und erreiche, dass deine Applikation den UseCase erfüllt.</b> 

**Tipp:** `app.run()` verwendet/blockiert bereits den Haupt-Thread. D.h. die `while True:`-Loop nicht im Haupt-Thread laufen, da dieser bereits beschäftigt ist. Deshalb muss der Scheduler aus einem anderen Thread aus gestartet werden. Dafür kannst du folgenden Code verwenden:  

In [30]:
import time
from threading import Thread
from flask import Flask
import schedule
def schedule_thread():
    while True:
        schedule.run_pending()
        time.sleep(1)


if __name__ == '__main__':
    thread_schedule = Thread(target=schedule_thread, args=())
    thread_schedule.start()
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Jan/2024 17:54:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2024 17:54:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2024 17:54:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Jan/2024 17:55:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2024 17:55:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Jan/2024 17:55:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2024 17:55:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [24/Jan/2024 17:55:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jan/2024 17:55:33] "GET /favicon.ico HTTP/1.1" 404 -


Updated last_state: OPEN
Updated last_state: CLOSED
